In [1]:
from torch.distributions.uniform import Uniform
from torch.distributions.beta import Beta
from torch.distributions.normal import Normal

import pickle
from utils import *
import torch.utils.data as data

from sklearn.datasets import make_moons

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import torch

/home/quan1691996/normalization_flow_basis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download dataset

In [2]:
FILENAME = 'shape.pkl'
with open(FILENAME, 'rb') as f:
    shape_data = pickle.load(f)
training_data, testing_data = shape_data['train'], shape_data['test']
# training_data.shape = (10479, 20, 20, 1)
training_data = (training_data > 127.5).astype(np.uint8)
# training_data.shape = (4491, 20, 20, 1)
testing_data = (testing_data > 127.5).astype(np.uint8)

train_loader = data.DataLoader(ShapesDataset(training_data), shuffle=True, batch_size=128)
test_loader = data.DataLoader(ShapesDataset(testing_data), shuffle=True, batch_size=128)

Training

In [3]:
flow = AutoRegressiveFlow(1, num_layers=5, n_components=10)
flow.apply(weights_init)

AutoRegressiveFlow(
  (model): Sequential(
    (0): MaskedConv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): MaskedConv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (5): ReLU()
    (6): MaskedConv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (7): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (8): ReLU()
    (9): MaskedConv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (10): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (11): ReLU()
    (12): MaskedConv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (13): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (14): ReLU()
    (15): MaskedConv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (16): MaskedConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (17): 

In [4]:
device = torch.device("cpu")

def train_and_eval(flow, epochs, lr, train_loader, test_loader, target_distribution):
    print('no of parameters is', sum(param.numel() for param in flow.parameters()))
    optimizer = torch.optim.Adam(flow.parameters(), lr=lr)
    train_losses, test_losses = [], []
    for epoch in range(epochs):
        print('Starting epoch:', epoch+1, 'of', epochs)
        train(flow, train_loader, optimizer, target_distribution)
        train_losses.append(eval_loss(flow, train_loader, target_distribution))
        test_losses.append(eval_loss(flow, test_loader, target_distribution))
    return flow, train_losses, test_losses

target_distribution = Uniform(torch.tensor(0).float().to(device),torch.tensor(1).float().to(device))
flow, train_losses, test_losses = train_and_eval(flow, 20, 1e-3, train_loader, test_loader, target_distribution)
print('train losses are', train_losses)
print('test losses are', test_losses)

no of parameters is 1013790
Starting epoch: 1 of 20


ValueError: Expected parameter loc (Tensor of shape (128, 10, 1, 20, 20)) of distribution Normal(loc: torch.Size([128, 10, 1, 20, 20]), scale: torch.Size([128, 10, 1, 20, 20])) to satisfy the constraint Real(), but found invalid values:
tensor([[[[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]]],


         [[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]]],


         [[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan,